# System resilience

Example of system resilience problem

At some point of accumulating evidences about object of study we stop just speculating about its properties and start mathematical modelling. 


In [ ]:
from matplotlib import rcParams, cycler
import matplotlib.pyplot as plt
import numpy as np

## Recap differential equations

There is a lot more that you can do with outputs (such as including interactive outputs)
with your book. For more information about this, see [the Jupyter Book documentation](https://jupyterbook.org)